In [1]:
from datetime import datetime, time, date
from calendar_ipynb.ipywidgets.calendar_selection import get_selection_from_cache as get_selected_calendars
from calendar_ipynb.events import fetch_events, get_primary_timezone, delete_and_duplicate_recurring_event_instance

target_date = "2025-09-07"

selected_calendars = get_selected_calendars()
if not selected_calendars:
    raise ValueError("No calendars selected")

primary_timezone = get_primary_timezone(selected_calendars=selected_calendars)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


Logging Setup!


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


In [2]:
from_datetime = datetime.combine(date.fromisoformat(target_date), time(0, 0, 0)).replace(tzinfo=primary_timezone)
to_datetime = datetime.combine(date.fromisoformat(target_date), time(23, 59, 59)).replace(tzinfo=primary_timezone)
events = []

for email, calendars in selected_calendars.items():
    for calendar_id in calendars:
        # Fetch events for the selected calendar
        events.extend(fetch_events(
            email=email,
            calendar_id=calendar_id,
            from_datetime=from_datetime,
            to_datetime=to_datetime,
        ))

INFO:calendar_ipynb.events:Fetching events from 2025-09-07T00:00:00+05:30 to 2025-09-07T23:59:59+05:30for fahimalizain@gmail.com
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events:Fetching events from 2025-09-07T00:00:00+05:30 to 2025-09-07T23:59:59+05:30for fahimalizain@gmail.com
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events:Fetching events from 2025-09-07T00:00:00+05:30 to 2025-09-07T23:59:59+05:30for fahimalizain@gmail.com
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events:Fetching events from 2025-09-07T00:00:00+05:30 to 2025-09-07T23:59:59+05:30for fahimalizain@gmail.com
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events:Fetching events from 2025-09-07T00:00:00+05:30 to 2025-09-07T23:59:59+05:30for fahim.zain@kwalee.

In [3]:
for event in events:
    if not event.get("recurringEventId", None):
        continue

    if event.get("status", None) != "confirmed":
        continue

    if event.get("eventType", None) not in ["default", "fromGmail"]:
        continue

    if "creator" in event and event["creator"].get("email", None) != event["email"]:
        continue

    delete_and_duplicate_recurring_event_instance(
        email=event['email'],
        calendar_id=event['calendar_id'],
        instance=event
    )

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events:Deleting and duplicating event instance: Woke Up
INFO:calendar_ipynb.events:Created new event: https://www.google.com/calendar/event?eid=MHZzZmQzaWpqYmxqNDdhanJyOGNqaDE1ZzAgZXN2YWc0ZmJiOHExOTJxM2tzaXBxaDVkMmdAZw
INFO:calendar_ipynb.events:Deleted original instance: https://www.google.com/calendar/event?eid=ZGMyYzc4OWRmMDhkNDc3YTkzMzUxNGI3OTdjMGQxNjJfMjAyNTA5MDZUMjMzMDAwWiBlc3ZhZzRmYmI4cTE5MnEza3NpcHFoNWQyZ0Bn
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events:Deleting and duplicating event instance: Prayer: Subah
INFO:calendar_ipynb.events:Created new event: https://www.google.com/calendar/event?eid=NDMxZzd1Z3Y1ZWo2NTRwNW1iNHNlZWxrOGcgZXN2YWc0ZmJiOHExOTJxM2tzaXBxaDVkMmdAZw
INFO:calendar_ipynb.events:Deleted original instance: https://www.google.com/calendar/event?eid=a3JiYXE1cTVmZGFoZThlNDV1bG1jaW0yZzdfMjA